In [1]:
import numpy as np
import pandas as pd
# import feedparser
import xmltodict
import requests
from bs4 import BeautifulSoup
import json
import copy
import os
import lxml

#### Urls to get data

In [2]:
root_url = 'https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/' #Test url
# root_url = "https://www.resultats-elections.interieur.gouv.fr/telechargements/PR2022/" #Production url
url_presidentielle = root_url + 'resultatsT1/'

#### Function used

In [3]:
def findNotInString(str, l):
    for x in l:
        if x in str:
            return False
    return True

to_filter_in_url = ['../','index.xml', '?', '/avotreservice/']

def searchSize(s):
    if 'K' in s:
        return float(s.replace('K', ''))
    elif 'M' in s:
        return float(s.replace('M', ''))*1000

def findKeys(l_d):
    all_keys = list(l_d[0].keys())
    for x in l_d:
        for y in x.keys():
            if y not in all_keys:
                all_keys.append(y)
    return all_keys

dict_error_urls = []

def find_all_comm(url):
    df = pd.read_html(url)[0]
    df = df.iloc[:,:-1].dropna()
    df['size_num'] = df.Size.map(searchSize).astype('float')
    if df.Name.iloc[0][-1] == '/':
        xml_files = [findAllCommUrl2(url + x) for x in list(df.Name) if x[-1] == '/']
    else:
        xml_files = [url + df.sort_values('size_num', ascending=False).iloc[0].Name]
    return xml_files


def findAllCommUrl2(url):
    df = pd.read_html(url)[0]
    df = df.iloc[:,:-1].dropna()
    df['size_num'] = df.Size.map(searchSize).astype('float')
    return url + df.sort_values('size_num', ascending=False).iloc[0].Name

def parseThisUrl(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    T1 = soup.select('table')[0]
    those_urls = [url + x.get('href') for x in T1.select('tr a') if findNotInString(x.get('href'), to_filter_in_url)]
    return those_urls

#### All data url for communes

In [4]:
url_reg = [x for x in parseThisUrl(url_presidentielle) if '/' in x[-1]]

all_xml_files = [find_all_comm(x) for x in url_reg]

all_comm_xml_files = []
for x in all_xml_files:
    all_comm_xml_files = all_comm_xml_files + x

## Données communales

In [5]:
l_full = []
# l_m_dict = []
error_list = []
for i, dep in enumerate(all_comm_xml_files):
    print(i, dep)
    try:
        r = requests.get(dep)
        d = xmltodict.parse(r.text)
    except:
        print ('error')
        continue
    if 'html' in d.keys():
        print ('error')
        continue
    code_dep = d['Election']['Departement']['CodDpt']
    code_dep_3d = d['Election']['Departement']['CodDpt3Car']
    code_dep_min = d['Election']['Departement']['CodMinDpt']
    try:
        data_comm = d['Election']['Departement']['Communes']['Commune']
    except:
        data_comm = d['Election']['Departement']
    # print(data_comm)
    data_comm = data_comm if isinstance(data_comm, list) else [data_comm]
    for c in data_comm:
        d_comm = {}
        d_comm['CodSubCom'] = c['CodSubCom'] if code_dep != "ZZ" else "000"
        d_comm['LibSubCom'] = c['LibSubCom'] if code_dep != "ZZ" else "Français  de l'etranger"
        m =  c['Tours']['Tour']['Mentions']
        d_comm['Inscrits'] = m['Inscrits']['Nombre']
        d_comm['Abstentions'] = m['Abstentions']['Nombre']
        d_comm['Votants'] = m['Votants']['Nombre']
        d_comm['Blancs'] = m['Blancs']['Nombre']
        d_comm['Nuls'] = m['Nuls']['Nombre']
        d_comm['Exprimes'] = m['Exprimes']['Nombre']
        this_sel = c['Tours']['Tour']['Resultats']['Candidats']['Candidat']
        if isinstance(this_sel, list)  == False:
            this_sel = [this_sel]
        for r in this_sel:
            d3 = copy.deepcopy(d_comm)
            d3['NomPsn'] = r['NomPsn']
            d3['NumPanneauCand'] = r['NumPanneauCand']
            d3['NbVoix'] = r['NbVoix']
            d3['codeDepartement'] = code_dep_3d
            d3['code_dep_min'] = code_dep_min
            d3['code_dep'] = code_dep
#             tete_liste = r['CandidatsListe']['Candidat'][0]
#             d3['tete_liste'] = tete_liste['CivilitePsn'] + ' ' + tete_liste['PrePsn'] + ' ' + tete_liste['NomPsn']
            l_full.append(d3)
df_all_communes = pd.DataFrame(l_full)

0 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/099/099.xml
1 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/975/975com.xml
2 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/977/977com.xml
3 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/986/986com.xml
4 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/987/987com.xml
5 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/988/988com.xml
6 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/001/971/971com.xml
7 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/002/972/972com.xml
8 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/003/973/973com.xml
9 ht

In [6]:
df_all_communes

,CodSubCom,LibSubCom,Inscrits,Abstentions,Votants,Blancs,Nuls,Exprimes,NomPsn,NumPanneauCand,NbVoix,codeDepartement,code_dep_min,code_dep
0,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,HORTALA,1,5520,099,99,ZZ
1,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,MIREPOIX,2,8227,099,99,ZZ
2,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,ARBOCOL,3,13638,099,99,ZZ
3,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,GIRARD,4,2120,099,99,ZZ
4,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,SANJUAN,5,57746,099,99,ZZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420967,366,Chisa,177,50,127,3,3,121,JONCOUR,8,0,02B,2B,2B
420968,366,Chisa,177,50,127,3,3,121,POMPANON,9,35,02B,2B,2B
420969,366,Chisa,177,50,127,3,3,121,BAVÉLÈRE,10,31,02B,2B,2B
420970,366,Chisa,177,50,127,3,3,121,MÉDÉRIC,11,10,02B,2B,2B


#### Departements avec arrondissement

In [8]:
df_all_communes[df_all_communes.CodSubCom.str.contains('AR')].code_dep.unique()

array(['75', '69', '13'], dtype=object)

## Donnees regionales

In [9]:
def find_all_reg(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return url + [x.getText() for x in soup.select('tr td a') if '.xml' in x.getText()][0]

folders_regions_otm = [x for x in parseThisUrl(url_reg[0]) if '/' in x[-1]] + url_reg[1:]

urls_data_region = [find_all_reg(x) for x in folders_regions_otm]

In [10]:
l_full_reg = []
# l_m_dict = []
error_list = []
for i, reg in enumerate(urls_data_region):
    print(i, reg)
    try:
        r = requests.get(reg)
        d = xmltodict.parse(r.text)
    except:
        print ('error')
        continue
    if 'html' in d.keys():
        print ('error')
        continue
    if 'Region' in d['Election'].keys():
        base_reg = d['Election']['Region']
        code_reg = base_reg['CodReg']
        code_reg_3d = base_reg['CodReg3Car']
        LibReg = base_reg['LibReg']
    else:
        base_reg = d['Election']['Departement']
        code_reg = base_reg['CodDpt']
        code_reg_3d = base_reg['CodDpt3Car']
        LibReg = base_reg['LibDpt']
    data_reg = {}
    m =  base_reg['Tours']['Tour']['Mentions']
    data_reg['Inscrits'] = m['Inscrits']['Nombre']
    data_reg['Abstentions'] = m['Abstentions']['Nombre']
    data_reg['Votants'] = m['Votants']['Nombre']
    data_reg['Blancs'] = m['Blancs']['Nombre']
    data_reg['Nuls'] = m['Nuls']['Nombre']
    data_reg['Exprimes'] = m['Exprimes']['Nombre']
    this_sel = base_reg['Tours']['Tour']['Resultats']['Candidats']['Candidat']
    if isinstance(this_sel, list)  == False:
        this_sel = [this_sel]
    for r in this_sel:
        d3 = copy.deepcopy(data_reg)
        d3['NomPsn'] = r['NomPsn']
        d3['NumPanneauCand'] = r['NumPanneauCand']
        d3['NbVoix'] = r['NbVoix']
        d3['codeRegion3d'] = code_reg_3d
        d3['code_region'] = code_reg
        d3['lib_region'] = LibReg
#            tete_liste = r['CandidatsListe']['Candidat'][0]
#            d3['tete_liste'] = tete_liste['CivilitePsn'] + ' ' + tete_liste['PrePsn'] + ' ' + tete_liste['NomPsn']
        l_full_reg.append(d3)
df_all_reg = pd.DataFrame(l_full_reg)

0 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/099/099.xml
1 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/975/975.xml
2 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/977/977.xml
3 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/986/986.xml
4 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/987/987.xml
5 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/988/988.xml
6 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/001/001.xml
7 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/002/002.xml
8 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/003/003.xml
9 https://www.resultats-elections.interi

In [13]:
dict_new_code_reg = {'01' : 'ZA',
'02' : 'ZB',
'03' : 'ZC',
'04' : 'ZD',
'06' : 'ZM'}

df_all_reg['code_region'] = df_all_reg.code_region.map(lambda x: dict_new_code_reg[x] if x in dict_new_code_reg.keys() else x)

for col in ['Inscrits', 'Abstentions', 'Votants', 'Blancs', 'Nuls', 'Exprimes', 'NbVoix']:
    df_all_reg[col] = df_all_reg[col].astype('int')

df_all_reg['part_votes'] = df_all_reg.apply(lambda row: np.round(100*row.NbVoix / row.Exprimes, 1), 1)

index_columns_reg = ['Inscrits', 'Abstentions', 'Votants', 'Blancs', 'Nuls', 'Exprimes', 'codeRegion3d', 
                                  'code_region', 'lib_region']
pivoted_reg = pd.pivot_table(df_all_reg.drop('NumPanneauCand', axis=1), index=index_columns_reg, columns='NomPsn')

pivoted_flattened_voix_reg = pivoted_reg['NbVoix'].reset_index()
pivoted_flattened_votes_reg = pivoted_reg['part_votes'].reset_index()

pivoted_flattened_reg = pd.merge(pivoted_flattened_voix_reg, pivoted_flattened_votes_reg, on=index_columns_reg, suffixes=('_voix', '_partVotes'))

df_reg_formap = pivoted_flattened_voix_reg.copy()

df_reg_formap.columns = [x if x not in dict_maping_candidats.keys() else dict_maping_candidats[x] for x in df_reg_formap.columns]

In [ ]:
df_reg_formap

## Données départementales

In [14]:
def find_all_dep(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return url + [x.getText() for x in soup.select('tr td a') if '.xml' in x.getText() and
            len(x.getText()) <= 8][0]

In [15]:
folders_dep_otm = []
for r in url_reg:
    folders_dep_otm += [x for x in parseThisUrl(r) if '.xml' not in x]

urls_data_departements = [find_all_dep(x) for x in folders_dep_otm]

In [16]:
l_full_dep = []
# l_m_dict = []
error_list = []
for i, dep in enumerate(urls_data_departements):
    print(i, dep)
    try:
        r = requests.get(dep)
        d = xmltodict.parse(r.text)
    except:
        print ('error')
        continue
    if 'html' in d.keys():
        print ('error')
        continue
    base_dep = d['Election']['Departement']
    code_dep = base_dep['CodDpt']
    code_dep_3d = base_dep['CodDpt3Car']
    LibDep = base_dep['LibDpt']
    data_dep = {}
    m =  base_dep['Tours']['Tour']['Mentions']
    data_dep['Inscrits'] = m['Inscrits']['Nombre']
    data_dep['Abstentions'] = m['Abstentions']['Nombre']
    data_dep['Votants'] = m['Votants']['Nombre']
    data_dep['Blancs'] = m['Blancs']['Nombre']
    data_dep['Nuls'] = m['Nuls']['Nombre']
    data_dep['Exprimes'] = m['Exprimes']['Nombre']
    this_sel = base_dep['Tours']['Tour']['Resultats']['Candidats']['Candidat']
    if isinstance(this_sel, list)  == False:
        this_sel = [this_sel]
    for r in this_sel:
        d3 = copy.deepcopy(data_dep)
        d3['NomPsn'] = r['NomPsn']
        d3['NumPanneauCand'] = r['NumPanneauCand']
        d3['NbVoix'] = r['NbVoix']
        d3['codeDepartement3d'] = code_dep_3d
        d3['code_departement'] = code_dep
        d3['lib_departement'] = LibDep
#            tete_liste = r['CandidatsListe']['Candidat'][0]
#            d3['tete_liste'] = tete_liste['CivilitePsn'] + ' ' + tete_liste['PrePsn'] + ' ' + tete_liste['NomPsn']
        l_full_dep.append(d3)
df_all_dep = pd.DataFrame(l_full_dep)

0 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/099/099.xml
1 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/975/975.xml
2 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/977/977.xml
3 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/986/986.xml
4 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/987/987.xml
5 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/000/988/988.xml
6 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/001/971/971.xml
7 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/002/972/972.xml
8 https://www.resultats-elections.interieur.gouv.fr/telechargements/EssaiPR2022/resultatsT1/003/973/973.xml
9 https://www.resultats-elec

In [17]:
for col in ['Inscrits', 'Abstentions', 'Votants', 'Blancs', 'Nuls', 'Exprimes', 'NbVoix']:
    df_all_dep[col] = df_all_dep[col].astype('int')

df_all_dep['part_votes'] = df_all_dep.apply(lambda row: np.round(100*row.NbVoix / row.Exprimes, 1), 1)

index_columns = ['Inscrits', 'Abstentions', 'Votants', 'Blancs', 'Nuls', 'Exprimes', 'codeDepartement3d', 
                                  'code_departement', 'lib_departement']
pivoted = pd.pivot_table(df_all_dep.drop('NumPanneauCand', axis=1), index=index_columns, columns='NomPsn')

pivoted_flattened_voix = pivoted['NbVoix'].reset_index()
pivoted_flattened_votes = pivoted['part_votes'].reset_index()

pivoted_flattened_departements = pd.merge(pivoted_flattened_voix, pivoted_flattened_votes, on=index_columns, suffixes=('_voix', '_partVotes'))

candidats_2022 = ['MACRON', 'MÉLENCHON', 'LE PEN', 'PÉCRESSE',  'ZEMMOUR', 'JADOT', 'ROUSSEL', 'HIDALGO', 'POUTOU', 'ARTHAUD', 'DUPONT-AIGNAN', 'LASSALLE']

In [18]:
df_dep_formap = pivoted_flattened_voix.copy()

df_dep_formap.columns = [x if x not in dict_maping_candidats.keys() else dict_maping_candidats[x] for x in df_dep_formap.columns]

In [51]:
df_dep_formap.to_csv('coronavirus/apps/election-presidentielle-2022/data/election_data_dep.csv', index=False)

In [ ]:
df_dep_formap

#### Sauvegarde des données communales

In [19]:
df_all_communes

,CodSubCom,LibSubCom,Inscrits,Abstentions,Votants,Blancs,Nuls,Exprimes,NomPsn,NumPanneauCand,NbVoix,codeDepartement,code_dep_min,code_dep
0,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,HORTALA,1,5520,099,99,ZZ
1,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,MIREPOIX,2,8227,099,99,ZZ
2,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,ARBOCOL,3,13638,099,99,ZZ
3,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,GIRARD,4,2120,099,99,ZZ
4,000,Français de l'etranger,1335453,395792,939661,24862,18406,896393,SANJUAN,5,57746,099,99,ZZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420967,366,Chisa,177,50,127,3,3,121,JONCOUR,8,0,02B,2B,2B
420968,366,Chisa,177,50,127,3,3,121,POMPANON,9,35,02B,2B,2B
420969,366,Chisa,177,50,127,3,3,121,BAVÉLÈRE,10,31,02B,2B,2B
420970,366,Chisa,177,50,127,3,3,121,MÉDÉRIC,11,10,02B,2B,2B


In [22]:
for col in ['Inscrits', 'Abstentions', 'Votants', 'Blancs', 'Nuls', 'Exprimes', 'NbVoix']:
    df_all_communes[col] = df_all_communes[col].astype('int')

df_all_communes['part_votes'] = df_all_communes.apply(lambda row: np.round(100*row.NbVoix / row.Exprimes, 1), 1)

index_columns_communes = ['CodSubCom', 'LibSubCom', 'Inscrits', 'Abstentions', 'Votants', 'Blancs', 'Nuls', 'Exprimes',
                 'codeDepartement', 'code_dep_min', 'code_dep']
pivoted_communes = pd.pivot_table(df_all_communes.drop('NumPanneauCand', axis=1), index=index_columns_communes, columns='NomPsn')

pivoted_flattened_voix_communes = pivoted_communes['NbVoix'].reset_index()

In [29]:
df_all_communes['code_commune'] = df_all_communes.apply(lambda row: row['code_dep_min'] + row['CodSubCom'], 1)